In [1]:
import pandas as pd
from zipfile import ZipFile
from  pathlib import Path
import calendar
import datetime
import pickle

In [2]:
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')


In [5]:
example.columns


Index(['product_lvls', 'category', 'demo_lvls', 'socdem_gr',
       'time_period_type', 'year', 'period_lbl', 'buying_households',
       'raw_shoppers', 'loyalty_volume_based',
       'loyalty_value_based_local_currency', 'loyalty_value_based_usd',
       'percent_2__time_buyers', 'percent_household_penetration',
       'purchase_frequency', 'occasions', 'item_buying_rate_local_currency',
       'item_buying_rate_usd', 'purchase_size_local_currency',
       'purchase_size_usd', 'purchase_size_su',
       'average_per_su_local_currency', 'average_per_su_usd', 'volume_su',
       'volume_physical_units', 'spend_local_currency', 'spend_usd',
       'product_hier', 'demo_hier', 'products', 'features', 'cat_segment',
       'demo_groups'],
      dtype='object')

In [48]:
path = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\codes\periods.xlsx')

In [2]:
new_data =pd.read_parquet('data/new_data_merged/new_data.pq')
col_check = {}
for col in new_data.columns:
    if new_data[col].isna().sum():
        col_check[col] = new_data[col].isna().sum()
print(col_check)
      

{}


In [3]:
meta_information = new_data[['category','period_lbl', 'demo_groups','shop_code']].drop_duplicates()
meta_information = meta_information.sort_values(by = ['category','period_lbl', 'demo_groups','shop_code']).reset_index(drop=True)



In [5]:
new_data.columns

Index(['spend_local_currency', 'volume_su', 'volume_physical_units',
       'occasions', 'buying_households', 'raw_shoppers',
       'percent_household_penetration', 'purchase_frequency',
       'item_buying_rate_local_currency', 'purchase_size_local_currency',
       'purchase_size_su', 'average_per_su_local_currency', 'year', 'features',
       'category', 'products', 'product_hier', 'product_lvls', 'demo_groups',
       'demo_hier', 'demo_lvls', 'socdem_gr', 'cat_segment', 'shop_code',
       'shop_lvls', 'channel_code', 'file', 'period_lbl', 'time_period_type'],
      dtype='object')

In [55]:
meta_2 = new_data.groupby(['category','period_lbl', 'demo_groups','shop_code'])['file'].agg(set)

In [62]:
meta_2['set'].map(len).value_counts()

set
1    19941
2       84
Name: count, dtype: int64

In [61]:
meta_2[meta_2['set'].map(len)==2]

set  \
category period_lbl demo_groups shop_code                                                      
1        108        1           1          {out_datatile/diapers_socdem-new24.parquet, ou...   
         109        1           1          {out_datatile/diapers_socdem-new24.parquet, ou...   
         117        1           1          {out-for-datatile-sep24/diapers_allshops.parqu...   
         119        1           1          {out-for-datatile-sep24/diapers_allshops.parqu...   
2        107        1           1          {out-for-datatile/br_allshops.parquet, out_dat...   
...                                                                                      ...   
6        115        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         116        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         117        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         119        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   
         120        1           1          {out-for-datatile-sep24/deterg_allshops.parque...   

                                            len  
category period_lbl demo_groups shop_code        
1        108        1           1           650  
         109        1           1           618  
         117        1           1           654  
         119        1           1           620  
2        107        1           1           684  
...                                         ...  
6        115        1           1          2614  
         116        1           1          2484  
         117        1           1          2618  
         119        1           1          2510  
         120        1           1          2396  

[84 rows x 2 columns]

In [20]:
meta_2 = pd.DataFrame(meta_2)

In [30]:
meta_2['len'] = meta_2['file'].str.(',')

In [45]:
meta_2.loc[1]

file  \
period_lbl demo_groups shop_code                                                      
9          1           1          [out-for-datatile/diapers_allshops.parquet, ou...   
                       2          [out-for-datatile/diapers_allshops.parquet, ou...   
                       4          [out-for-datatile/diapers_allshops.parquet, ou...   
                       5          [out-for-datatile/diapers_allshops.parquet, ou...   
                       8          [out-for-datatile/diapers_allshops.parquet, ou...   
...                                                                             ...   
119        49          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           50          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           51          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           52          1          [out_datatile/diapers_socdem-new24.parquet, ou...   
           53          1          [out_datatile/diapers_socdem-new24.parquet, ou...   

                                  len  
period_lbl demo_groups shop_code       
9          1           1          NaN  
                       2          NaN  
                       4          NaN  
                       5          NaN  
                       8          NaN  
...                               ...  
119        49          1          NaN  
           50          1          NaN  
           51          1          NaN  
           52          1          NaN  
           53          1          NaN  

[986 rows x 2 columns]

In [73]:
with open('data/tmp/old_data.pickle', 'rb') as f:
    old_data = pickle.load(f)

In [74]:
df_union = pd.concat([old_data, new_data], ignore_index=True)

In [65]:
i = df_union.shape
i

(13857113, 37)

In [66]:
df_union_direct_drop = df_union.drop_duplicates()

In [67]:
df_union_direct_drop.shape

(13857113, 37)

In [68]:
example = pd.read_excel(r'.\codes\Example.xlsx', sheet_name = 'example')
fin_cols = example.columns.to_list() + ['shop_code', 'shop_lvls','channel_code']


In [69]:
df_union_list_drop = df_union.drop_duplicates(subset = fin_cols )

In [71]:
i[0] -df_union_list_drop.shape[0]

1985

In [72]:
fin_cols

['product_lvls',
 'category',
 'demo_lvls',
 'socdem_gr',
 'time_period_type',
 'year',
 'period_lbl',
 'buying_households',
 'raw_shoppers',
 'loyalty_volume_based',
 'loyalty_value_based_local_currency',
 'loyalty_value_based_usd',
 'percent_2__time_buyers',
 'percent_household_penetration',
 'purchase_frequency',
 'occasions',
 'item_buying_rate_local_currency',
 'item_buying_rate_usd',
 'purchase_size_local_currency',
 'purchase_size_usd',
 'purchase_size_su',
 'average_per_su_local_currency',
 'average_per_su_usd',
 'volume_su',
 'volume_physical_units',
 'spend_local_currency',
 'spend_usd',
 'product_hier',
 'demo_hier',
 'products',
 'features',
 'cat_segment',
 'demo_groups',
 'shop_code',
 'shop_lvls',
 'channel_code']

In [79]:
meta_2 = df_union.groupby(['cat_segment','category','period_lbl', 'demo_groups','shop_code'])['file'].agg(set)

In [87]:
meta_2.map(len).value_counts()

file
1    169624
2      2257
3         2
Name: count, dtype: int64

In [86]:
meta_2.loc[meta_2.map(len)>1]

cat_segment  category  period_lbl  demo_groups  shop_code
100          1         9           1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       12          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       15          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       18          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                       21          1            1            {PG_Flatfiles_Diapers, out-for-datatile/diaper...
                                                                                   ...                        
611          6         115         1            1            {out-for-datatile-sep24/deterg_allshops.parque...
                       116         1            1            {out-for-datatile-sep24/deterg_allshops.parque...
                       117         1            1     

In [1]:
pg_db = """'JUICE'::text, 'ICE TEA'::text, 'ICE COFFEE'::text, 'CARBONATED SOFT DRINKS'::text, 'KVASS'::text, 'ENERGY DRINKS'::text, 'INSTANT SOFT DRINKS'::text, 'WATER'::text, 'KISEL'::text, 'BEER'::text, 'MEAD'::text, 'VODKA'::text, 'WINE'::text, 'CHAMPAGNE'::text, 'ALCOHOL STRONG'::text, 'ALCOHOL COCKTAILS'::text, 'CIDER'::text, 'ROASTED COFFEE'::text, 'INSTANT COFFEE'::text, 'COFFEE SUBSTITUTE'::text, 'TEA'::text, 'INSTANT TEA'::text, 'CHOCOLATE DRINKS'::text, 'MEAT PATE'::text, 'SAUSAGES'::text, 'SAUSAGE'::text, 'MEAT DELICACIES'::text, 'MEAT / MEAT SEMI-FINISHED'::text, 'POTATO PUREE'::text, 'FROZEN VEGETABLES'::text, 'FROZEN FRUITS'::text, 'FROZEN FLOURY PRODUCTS'::text, 'DOUGH'::text, 'PIZZA'::text, 'FROZEN READY-TO-EAT MEALS'::text, 'EDIBLE OIL'::text, 'LARD'::text, 'BUTTER'::text, 'MARGARINE/MIX'::text, 'PASTA'::text, 'INSTANT NOODLES'::text, 'TRADITIONAL CEREALS (KRUPA)'::text, 'TRADITIONAL CEREALS (RICE)'::text, 'FLOUR'::text, 'FAST-COOKING CEREALS'::text, 'INSTANT CEREALS'::text, 'BREAKFAST CEREALS'::text, 'HEALTHY SNACK BARS'::text, 'SAVOURY_BOUILLONS'::text, 'INSTANT SOUPS'::text, 'READY-TO-EAT SOUPS'::text, 'COOKING SOUPS'::text, 'READY-TO-EAT SALADS'::text, 'READY-TO-EAT MEALS'::text, 'FRUIT JELLY/DESSERTS'::text, 'COOKING ADDITIVES DRY'::text, 'COOKING ADDITIVES WET'::text, 'SAVOURY_SEASONINGS'::text, 'SAVOURY_MEALMAKERS'::text, 'SALT'::text, 'KETCHUP'::text, 'MUSTARD'::text, 'ADZHIKA'::text, 'HORSE-RADISH'::text, 'MAYONNAISE'::text, 'SAUCE'::text, 'SYRUP'::text, 'PROCESSED CHEESE'::text, 'CONDENSED MILK'::text, 'SPREAD CHEESE'::text, 'YELLOW CHEESE'::text, 'WHITE CHEESE'::text, 'CHOCOLATE TABLETS'::text, 'CHOCOLATE BARS'::text, 'CHEWING MARMALADE'::text, 'CHOCOLATE CONFETTI'::text, 'CHOCOLATE IN BOXES'::text, 'CHOCOLATE OTHER'::text, 'SWEET SPREADS'::text, 'ZEPHYR'::text, 'PASTILA'::text, 'MARMALADE/ORIENTAL SWEETS'::text, 'SUGAR CONFECTIONERY'::text, 'ICE CREAM'::text, 'PASTRY_PECHENIE'::text, 'PASTRY_CHILLED SNACKS'::text, 'PASTRY_PIROZHNYE LONG SHELF'::text, 'PASTRY_PIROZHNYE FRESH'::text, 'PASTRY_CROISSANTS LONG SHELF'::text, 'PASTRY_TORTY LONG SHELF'::text, 'PASTRY_TORTY FRESH'::text, 'PASTRY_PIROGI LONG SHELF'::text, 'PASTRY_PIROGI FRESH'::text, 'PASTRY_RULETY LONG SHELF'::text, 'PASTRY_RULETY FRESH'::text, 'PASTRY_KEKSY/TORTIN.LONG SHELF'::text, 'PASTRY_KEKSY FRESH'::text, 'PASTRY_PRYANIKI'::text, 'PASTRY_SUKHARI'::text, 'PASTRY_SUSHKI/BARANKI'::text, 'PASTRY_SOLOMKA/PALOCHKI/KREND.'::text, 'PASTRY_ORIENTAL SWEETS'::text, 'PASTRY_TARTALETKI'::text, 'WAFERS'::text, 'BREAD'::text, 'CRISPBREAD'::text, 'SALTY BISCUITS'::text, 'SALTY NUTS'::text, 'SALTY STICKS & PRETZELS'::text, 'SALTY BREAD STICKS'::text, 'POTATO CHIPS'::text, 'EXTRUDED SNACKS'::text, 'CROUTONS'::text, 'POP-CORN'::text, 'SEMECHKI'::text, 'SUGAR'::text, 'CRABS/SURIMI'::text, 'HERRING'::text, 'SALTY SQUIDS'::text, 'FISH FRESH&FROZEN'::text, 'FISH SALTED&SMOKED'::text, 'FISH SEMI-FINISHED'::text, 'SEAFOOD'::text, 'IKRA / FISH BUTTER'::text, 'MILK WHITE'::text, 'MILK BAKED'::text, 'MILK FLAVOURED'::text, 'KEFIR'::text, 'RYAZHENKA & VARENETS'::text, 'PROSTOKVASHA'::text, 'MILK DRINKS'::text, 'YOGHURT SPOONABLE'::text, 'YOGHURT LIQUID'::text, 'MILK DESSERTS'::text, 'POWDERED CREAMS'::text, 'SOUR CREAM (SMETANA)'::text, 'SWEET CREAM (SLIVKI)'::text, 'WHIPPED CREAM'::text, 'TVOROG'::text, 'GLAZIROVANNY SYROK'::text, 'SYR DOMASHNIY'::text, 'TVOROZHNY SYROK'::text, 'ZERNENY TVOROG'::text, 'TVOROZHOK'::text, 'TVOROZHNAYA MASSA'::text, 'POULTRY'::text, 'EGGS'::text, 'CANNED VEGETABLES'::text, 'CANNED FRUITS'::text, 'JAM'::text, 'HONEY'::text, 'CANNED FISH'::text, 'CANNED MEAT'::text, 'CANNED SEAPRODUCTS'::text, 'FRESH VEGETABLES'::text, 'FRESH SALADS&HERBS'::text, 'FRESH FRUITS'::text, 'DRIED FRUITS & NUTS'::text, 'CHEWING GUM'::text, 'BABY FOOD'::text, 'BABY JUICE'::text """.replace('::text','')

In [9]:
pg_dbs = [x.strip() for  x in  pg_db.replace("'",'').split(',')]
print(*pg_dbs, sep ='\n')

JUICE
ICE TEA
ICE COFFEE
CARBONATED SOFT DRINKS
KVASS
ENERGY DRINKS
INSTANT SOFT DRINKS
WATER
KISEL
BEER
MEAD
VODKA
WINE
CHAMPAGNE
ALCOHOL STRONG
ALCOHOL COCKTAILS
CIDER
ROASTED COFFEE
INSTANT COFFEE
COFFEE SUBSTITUTE
TEA
INSTANT TEA
CHOCOLATE DRINKS
MEAT PATE
SAUSAGES
SAUSAGE
MEAT DELICACIES
MEAT / MEAT SEMI-FINISHED
POTATO PUREE
FROZEN VEGETABLES
FROZEN FRUITS
FROZEN FLOURY PRODUCTS
DOUGH
PIZZA
FROZEN READY-TO-EAT MEALS
EDIBLE OIL
LARD
BUTTER
MARGARINE/MIX
PASTA
INSTANT NOODLES
TRADITIONAL CEREALS (KRUPA)
TRADITIONAL CEREALS (RICE)
FLOUR
FAST-COOKING CEREALS
INSTANT CEREALS
BREAKFAST CEREALS
HEALTHY SNACK BARS
SAVOURY_BOUILLONS
INSTANT SOUPS
READY-TO-EAT SOUPS
COOKING SOUPS
READY-TO-EAT SALADS
READY-TO-EAT MEALS
FRUIT JELLY/DESSERTS
COOKING ADDITIVES DRY
COOKING ADDITIVES WET
SAVOURY_SEASONINGS
SAVOURY_MEALMAKERS
SALT
KETCHUP
MUSTARD
ADZHIKA
HORSE-RADISH
MAYONNAISE
SAUCE
SYRUP
PROCESSED CHEESE
CONDENSED MILK
SPREAD CHEESE
YELLOW CHEESE
WHITE CHEESE
CHOCOLATE TABLETS
CHOCOLATE BARS
CH